# Automate IOS-XR Zero Touch Provisioning

## Overview

On-boarding devices with the initial configuration (day-0 configuration) is a task which needs to be automated in many use-cases, especially if a large number of devices are involved. By limiting the human interaction in this process as much as possible service providers can reduce “truck-offload to service-activation” time, avoid to send skilled personal to the field to configure devices manually and achieve better deployment predictability.

For the management of day-1...day-n configurations we usually use the Cisco Network Services Orchestrator (NSO), so in the majority of use-cases the main goal of the day-0 configuration would be to bootstrap the devices and provide them with the minimal configuration needed to be able to onboard them in Cisco NSO - this means basically setting an IP address, user, password, generate some keys. Of course, we are able to manage more complex configurations if the use-case demands it, but it is recommended to manage as much as possible from the configuration using Cisco NSO, if available. 

Note that if the NED supports sending CLI commands to the device (using the life-status actions) it is also possible and straightforward to implement the MOPs (using workflows or scripts) which will use NSO to perform a software upgrade on the device to the desired version.

There are 3 main protocols supported by Cisco products for automating the day-0 configuration provisioning, depending on the type of device:

•	Cisco ZTP (Zero Touch Provisioning) - available on IOS-XR devices

•	Cisco PnP (Plug and Play) - available on branch devices (IOS/IOS-XE devices)

•	POAP (PowerOn Auto Provisioning) - available on the Nexus switches

This guide demonstrates the use of Zero Touch Provisioning Automation in the context of Cisco IOS-XR. There is also a brief description of the other protocols at the end of the lab guide as well as an example of using the PNP protocol with the NSO PNP server for an IOS-XE router.


## Cisco ZTP

<img src="img/ztp_1.png" width="800"/>

On IOS-XR, ZTP process starts at boot time and scans the configuration for the presence of a username. 
If there is no username configured, ZTP starts a DHCP client on the management interface for IPv4 and IPv6 simultaneously, and wait for a response. Together with the IP address the DHCP server will provide an URL to download a config file or a script using option 67 (or 59 for IPv6). We are using http URLs but other protocols like TFTP are also supported. 

The ZTP process will download the file and based on the first line of the text file received will: 
•	apply the configuration commands it contains (if it starts with ”!! IOS XR”)
•	assume this is a shell script and start the execution  (if it starts with ”#!/bin/bash” or ”#!/bin/sh”) 
•	start the execution if it is python script

If the text file cannot be interpreted as a script or a configuration file, or is bigger than 100MB, ZTP will erase the file and terminate its execution.

Using the different DHCP options/functions available on the server it is possible to dynamically create the file URL based on specific device information like the serial number. In this way we will be able to identify the device which made the request on the server side and provide a customized day0 configuration/script.

Using a script is the most flexible solution since inside the script it is possible to download other config files and apply them (using curl or wget for example / python libraries), implement logic, install packages and so on. In order to notify the provisioning server that the device has finished applying its configuration usually we make a HTTP request as the last command in the script which will signalize to our server that the device finished applying the config.

> The description in this section relates to Classic ZTP process
> supported on IOS-XR. In addition, IOS-XR Release 7 introduces a
> enhanced process called Secure ZTP (defined by Yang-Models to
> implement interactions between the router and a RESTCONF based ZTP
> Server) as well as support for YANG XML payload in the ZTP files,
> these enhancements are out of the scope of the current lab.

## LAB Details

This section includes the existing components of the dCloud session used for the lab. Th IP addresses and credentials of all the components used
may be found in the table below and in the Servers tab of the dCloud session:
 
 
| Name | Description | IP Address | Username | Password |
|--|--|--|--|--|
| Workstation | Windows 7 Server with Remote Desktop | 198.18.133.252 | Administrator | C1sco12345 |
| NSO | NSO	Cisco NSO Server | 198.18.134.50 | cisco | cisco |
| DHCP Server | ISC DHCP Server | 198.18.1.30 | cisco | cisco |
| XR9Kv | Cisco IOS-XR | 198.18.1.32 | cisco | cisco |
| CSR1Kv | Cisco IOS-XE | 198.18.1.31 | cisco | cisco |

<img src="img/ztp_topo_cl.png" width="800"/>

It will enable to understand and use ZTP on IOS-XR devices, to develop a prototype application in Python supporting automatic onboarding of XR devices in Cisco NSO as well as updating a Webex Team space about the onboarding status.

Specific focus will be provided on:

•	IOS-XR ZTP process and scripting capabilities

•	ISC DHCP server configuration

•	Cisco NSO actions capabilities

•	Cisco NSO notification capabilities

•	HTTP server Python application development

•	NETCONF/REST Python application development 

•	Webex Team REST API


<img src="img/ztp_cf.png" width="1000"/>